In [2]:
import pandas as pd
import numpy as np
data=pd.read_csv("origindata.csv")
print(data.index)
%matplotlib inline

RangeIndex(start=0, stop=1048575, step=1)


In [3]:
#Index(['flight', 'name', 'date', 'time'], dtype='object')
name_index=set(data["name"].values)
date_index=set(data["date"].values)
flight_index=set(data["flight"].values)
merged_data={}
merged_data["flight"]=[]
merged_data["date"]=[]
for i in name_index:
    merged_data[i]=[]
merged_data

{360.0: [],
 361.0: [],
 364.0: [],
 365.0: [],
 366.0: [],
 367.0: [],
 368.0: [],
 369.0: [],
 370.0: [],
 378.0: [],
 379.0: [],
 380.0: [],
 637.0: [],
 641.0: [],
 644.0: [],
 645.0: [],
 648.0: [],
 649.0: [],
 652.0: [],
 654.0: [],
 655.0: [],
 656.0: [],
 660.0: [],
 661.0: [],
 662.0: [],
 663.0: [],
 664.0: [],
 665.0: [],
 668.0: [],
 669.0: [],
 670.0: [],
 675.0: [],
 676.0: [],
 678.0: [],
 679.0: [],
 680.0: [],
 681.0: [],
 682.0: [],
 683.0: [],
 684.0: [],
 685.0: [],
 'date': [],
 'flight': []}

In [7]:
def str_to_time(str_temp):
    try:
        str_list=str_temp.split(":")
        return int(str_list[0])*60+int(str_list[1])
    except:
        return -1
    
print(1)
data_02=pd.read_csv("02.csv")
#日期转换为数字
# 
# Index(['NO', 'flight', 'date', '641 靠梯桥', '644 清洁完成', '645开货仓', '648 首件行李',
#        '649 最后一件行李', '655 配餐完成', '660 机务完成', '661 通知上客', '662登机开始', '668关货仓 ',
#        '669 撤轮档', '670 牵引车 机务放行', '360预计近机位时间', '361机务完成', '364 引导车到位-删除',
#        '365加油完成 缺失较多', '366牵引完成 缺失较多', '367推算预计起飞', '368开客舱', '369上轮档',
#        '370登机结束', '378 配餐开始 8个', '379清洁开始', '380撤梯桥', '637关客舱', '缺失个数'],
#       dtype='object')
index_list=list(data_02.columns)[3:-1]
print(data.columns)
x=data_02["641 靠梯桥"][1]
print(str_to_time(x))
new_cleaned_data={}
for node_name in index_list:
    data_temp=data_02[node_name]
    new_cleaned_data[node_name]=[]
    for i in range(len(data_02)):
        node_temp=str_to_time(data_temp[i])
        new_cleaned_data[node_name].append(node_temp)
        if i%10000==0:
            print(i)
new_cleaned_data["date"]=list(data_02["date"])
new_cleaned_data["flight"]=list(data_02["flight"])
new_cleaned_data["缺失个数"]=list(data_02["缺失个数"])
pd.DataFrame(new_cleaned_data).to_csv("cleaned02.csv")

1
Index(['flight', 'name', 'date', 'time'], dtype='object')
673
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
2000

In [9]:
#数据补齐，根据
data_03=pd.read_csv("cleaned02.csv")
#补齐方法，按照航班来，用同一个时刻的航班的数据进行补充
#设定字段补齐的优先级顺序，用哪些字段来查找
#那么第一步就是求字段两两之间的时间差关系分布，用总体数据来求
#